Import modules, including custom modules in /utils/ for stock aggregation and calculations

In [1]:
import numpy as np
import pandas as pd
import os

# Utility functions for calculations found in /utils/ folder
from utils.stock_assessments import get_asfis_mappings
from utils.species_landings import (
    format_fishstat,
    compute_species_landings,
    substitute_landings,
    add_species_landings,
)
from utils.stock_landings import compute_num_stocks, compute_landings
from utils.aggregate_tables import (
    compute_status_by_number,
    compute_weighted_percentages,
    compute_total_area_landings,
    compute_percent_coverage,
)

Define directories for imports

In [2]:
parent_dir = os.getcwd()
input_dir = os.path.join(parent_dir, "input")
clean_data_dir = os.path.join(parent_dir, "output/clean_data")

Read in the list of Area 37 stocks from the clean_data folder.

In [3]:
stock_assessments_complete = pd.read_excel(
    os.path.join(clean_data_dir, "stock_assessments.xlsx")
)

AREA = 37

area37_mask = stock_assessments_complete["Area"] == str(AREA)

stock_assessments = stock_assessments_complete[area37_mask].copy()

Compute the status by number

In [4]:
compute_status_by_number(stock_assessments, "Area").head(1)

,Area,No. of stocks,No. of U,No. of MSF,No. of O,No. of Sustainable,No. of Unsustainable,U (%),MSF (%),O (%),Sustainable (%),Unsustainable (%)
0,37,114,9,31,74,40,74,7.894737,27.192982,64.912281,35.087719,64.912281


We assign landings to the species based on values found in Fishstatj

In [5]:
# Retrieve fishstat data from input folder
fishstat = pd.read_csv(os.path.join(input_dir, "global_capture_production.csv"))

# Format fishstat data
mappings = get_asfis_mappings(input_dir, "ASFIS_sp_2024.csv")
asfis = mappings["ASFIS"]
code_to_scientific = dict(zip(asfis["Alpha3_Code"], asfis["Scientific_Name"]))

fishstat = format_fishstat(fishstat, code_to_scientific)

# Define dataframe for the species landings
species_landings = stock_assessments[
    ["FAO Major Fishing Area(s)", "ASFIS Scientific Name", "Location"]
].copy()

species_landings["FAO Area"] = species_landings["FAO Major Fishing Area(s)"].astype(int)
species_landings.drop(columns="FAO Major Fishing Area(s)", inplace=True)

# Compute species landings for all assessed stocks
year_start, year_end = 1950, 2021
years = list(range(year_start, year_end + 1))

pk = ["FAO Area", "ASFIS Scientific Name"]

sl_reduced = species_landings.drop_duplicates(pk)[pk].copy()

sl_reduced[years] = sl_reduced.apply(compute_species_landings, args=(fishstat,), axis=1)

We substitute the landings of proxy species for landings of species which have zero reported catch in Fishstatj. <br>

For Metapenaeus stebbingi, we use Metapenaeus monoceros. <br>

For Saurida lessepsianus and Saurida undosquamis, we use Synodontidae.

In [6]:
subs = [
    [AREA, ["Metapenaeus stebbingi"], ["Metapenaeus monoceros"]],
    [AREA, ["Saurida lessepsianus", "Saurida undosquamis"], ["Synodontidae"]],
]

sl_reduced = substitute_landings(sl_reduced, fishstat, subs, years)

Now we include the landings of Mullus spp (added to Mullus barbatus landings), Sardinella spp (added to Sardinella pilchardus landings), Trachurus spp (added to Trachurus mediterraneous landings). <br>

We haven't done this YET in the main branch since we are still considering for which stocks to perform this kind of operation on in other areas. Rest assured this will be performed in the final version, especially for this area.

In [7]:
# Dictionary keys are proxy species which landings are added to the species in the corresponding list, given the area
spl = {
    "Mullus spp": [AREA, "Mullus barbatus"],
    "Sardinella spp": [AREA, "Sardina pilchardus"],
    "Trachurus spp": [AREA, "Trachurus mediterraneus"],
}

sl_reduced = add_species_landings(sl_reduced, fishstat, spl, years)

# Merge computed landings back to list of all stocks
species_landings = pd.merge(species_landings, sl_reduced, on=pk)

# Only keep 2021 landings
species_landings = species_landings[
    ["ASFIS Scientific Name", "Location", "FAO Area", 2021]
]

species_landings = species_landings.rename(columns={2021: "Species Landings 2021"})

Now we pull the computed weights from the clean data folder. The stock weights are based on the "Catches" and "Landings" columns in the original Area 37 sheet (found in /input/updated_assessment_overview.xlsx, sheet name = "37"). The normalized weight for each stock is then computed from the reported catch or landings from the original file.

In [8]:
stock_weights = pd.read_excel(os.path.join(clean_data_dir, "stock_weights.xlsx"))

w_area_mask = stock_weights["FAO Area"] == AREA

stock_weights = stock_weights[w_area_mask]

stock_weights.drop(columns=["FAO Area", "Weight 1", "Weight 2"], inplace=True)

Based on the weights and the species landings, we can now compute the stock landings

In [9]:
primary_key = ["ASFIS Scientific Name", "Location"]

stock_landings = pd.merge(species_landings, stock_weights, on=primary_key)

# Add the status
stock_landings = pd.merge(stock_landings, stock_assessments, on=primary_key)

stock_landings = compute_num_stocks(stock_landings)

stock_landings["Stock Landings 2021"] = stock_landings.apply(compute_landings, axis=1)

Using the stock landings, we can compute the necessary statistics for Area 37! <br>

Let's start with the weighted percentages of status

In [10]:
compute_weighted_percentages(stock_landings, key="FAO Area").head(1)

Total Landings                                       \
                 U (Mt)  MSF (Mt)    O (Mt) Sustainable (Mt)   
FAO Area                                                       
37             0.042329  0.406536  0.356171         0.448865   

                            Weighted % by Landings                        \
         Unsustainable (Mt)                  U (%)    MSF (%)      O (%)   
FAO Area                                                                   
37                 0.356171               5.258024  50.499121  44.242854   

                                            
         Sustainable (%) Unsustainable (%)  
FAO Area                                    
37             55.757146         44.242854

Now let's compute the Total Area Landings <br>

We need the list of migratory tuna and migratory sharks in order to compute the total area landings. If we simply remove all tuna and sharks, we discount the local tuna and sharks (and the local tuna landings are nontrivial, as will be shown below).

In [11]:
# Define the ISSCAAP Groups to remove
isscaap_to_remove = [46, 61, 62, 63, 64, 71, 72, 73, 74, 81, 82, 83, 91, 92, 93, 94]

# Retrieve ISSCAAP Codes for fishstat and species landings
sn_to_isscaap = dict(zip(asfis["Scientific_Name"], asfis["ISSCAAP_Group_Code"]))
fishstat["ISSCAAP Code"] = fishstat["ASFIS Scientific Name"].map(sn_to_isscaap)
species_landings["ISSCAAP Code"] = species_landings["ASFIS Scientific Name"].map(
    sn_to_isscaap
)

species_landings["Area"] = species_landings["FAO Area"]

# Get list of migratory tuna and sharks
sharks_df = pd.read_excel(os.path.join(input_dir, "oceanic_pelagic_sharks.xlsx"))
sharks = sharks_df["ASFIS Scientific Name"].unique()

tuna_mask = stock_assessments_complete["Area"] == "Tuna"
tuna_df = stock_assessments_complete[tuna_mask].copy()
tuna = tuna_df["ASFIS Scientific Name"].unique()

special_groups = {"Sharks": sharks, "Tuna": tuna}

totl = compute_total_area_landings(
    AREA,
    fishstat,
    species_landings,
    special_groups=special_groups,
    isscaap_to_remove=isscaap_to_remove,
).loc[2021]

print(
    f"Area {AREA} Total Landings without migratory tuna or sharks: {totl/1e6:.4f} (Mt)"
)

Area 37 Total Landings without migratory tuna or sharks: 1.1088 (Mt)


Now, if we had simply taken out ISSCAAP Group 36, i.e. ALL tuna, including the local stocks, we would get a different number.

In [12]:
fs_isscaap_mask = ~fishstat["ISSCAAP Code"].isin(isscaap_to_remove + [36])
fs_area_mask = fishstat["Area"] == AREA

totl_naive = fishstat[fs_isscaap_mask & fs_area_mask][2021].sum()

print(f"Area {AREA} Total Landings without ALL tuna: {totl_naive/1e6:.4f} (Mt)")

Area 37 Total Landings without ALL tuna: 1.0802 (Mt)


This is because the local tuna landings are nontrivial in Area 37

In [21]:
fs_tuna_mask = fishstat["ISSCAAP Code"] == 36
fs_mig_tuna_mask = fishstat["ASFIS Scientific Name"].isin(tuna)

totl_local_tuna = (
    fishstat[fs_area_mask & fs_tuna_mask & ~fs_mig_tuna_mask]
    .groupby(["ASFIS Scientific Name", "ISSCAAP Code"])[2021]
    .sum()
    .sort_values(ascending=False)
)

totl_local_tuna.loc["Total"] = totl_local_tuna.sum()

totl_local_tuna.reset_index(name="2021 Landings")

,ASFIS Scientific Name,ISSCAAP Code,2021 Landings
0,Xiphias gladius,36.0,10082.15
1,Sarda sarda,36.0,6764.64
2,Euthynnus alletteratus,36.0,6471.03
3,Auxis rochei,36.0,3084.12
4,Scomberomorus commerson,36.0,1039.00
5,"Auxis thazard, A. rochei",36.0,922.53
6,Scombriformes (Scombroidei),36.0,200.00
7,Thunnini,36.0,33.09
8,Orcynopsis unicolor,36.0,27.41
9,Tetrapturus belone,36.0,26.24


Now, when we compute the total area landings for the percent coverage calculations, we do not exclude the landings from migratory tuna and sharks which are reported in the area.

In [14]:
totl_pc = compute_total_area_landings(
    AREA,
    fishstat,
    species_landings,
    isscaap_to_remove=isscaap_to_remove,
).loc[2021]

print(f"Area {AREA} Total Landings in 2021: {totl_pc/1e6:.4f} (Mt)")

Area 37 Total Landings in 2021: 1.1358 (Mt)


We compute the total assessed landings from the stock landings

In [15]:
totl_assessed = stock_landings["Stock Landings 2021"].sum()

print(f"Area {AREA} Total Assessed Landings in 2021: {totl_assessed/1e6:.4f} (Mt)")

Area 37 Total Assessed Landings in 2021: 0.8050 (Mt)


For percent coverage, we have to add back the landings from the migratory tunas and sharks which are assessed in the FAO Area. We will grab these landings from the clean data folder

In [16]:
stock_landings_full = pd.read_excel(
    os.path.join(clean_data_dir, "stock_landings_fao_areas.xlsx")
)

sl_tuna_mask = stock_landings_full["Area"] == "Tuna"
sl_sharks_mask = stock_landings_full["Area"] == "Sharks"
sl_area_mask = stock_landings_full["FAO Area"] == AREA

mig_stocks = stock_landings_full[(sl_tuna_mask | sl_sharks_mask) & sl_area_mask]

mig_l = mig_stocks["Stock Landings 2021"].sum()

print(f"Area {AREA} has {mig_l/1e6:.4f} (Mt) landings from migratory tuna and sharks.")

Area 37 has 0.0269 (Mt) landings from migratory tuna and sharks.


This gives us the percent coverage

In [17]:
pc = (totl_assessed + mig_l) / totl_pc

print(f"Area {AREA} percent coverage: {100 * pc:.2f}%")

Area 37 percent coverage: 73.24%


Which is the same value we get when using the utils function to compute percent coverage

In [19]:
# Merge the area stocks and the migratory landings into one dataframe
pc_stock_landings = pd.concat([stock_landings, mig_stocks])

pc_util = compute_percent_coverage(
    pc_stock_landings, species_landings, fishstat, isscaap_to_remove
)

pc_util.head(1)

,Area,Coverage (%)
0,37,73.243544
